In [1]:
"""

This whole section should be run in a jupyter notebook, one section after the other 



1) FILTERING FOR GENES OF INTEREST 
2) FIST RANKING OF GENES OF INTEREST 
3) ANNOTATING GENES OF INTEREST 



"""

'\n\nThis whole section should be run in a jupyter notebook, one section after the other \n\n\n\n1) FILTERING FOR GENES OF INTEREST \n2) FIST RANKING OF GENES OF INTEREST \n3) ANNOTATING GENES OF INTEREST \n\n\n\n'

In [10]:
"""
FILTERING FOR GENES OF INTEREST 

gwasgenesfile: file with genes of interest 
mergedrankedlist: input file, an rwr matrix 

filteredrankedlist: output file of filtered nodes 

"""
#
#
# filtering for GWAS GENES
#


goi=set()
fp=open(gwasgenesfile, 'r')
for line in fp:
    data= line.split('\t')
    genelist=data[1:]
    for what in genelist:
        goi.add(what)
fp.close()
print('%s genes in genes of interest' %len(goi))

goi_found=0
f_out=open(filteredrankedlist, 'w')
fp=open(mergedrankedlist, 'r')
for line in fp:
    if 'entrez_id' in line:
        f_out.write('%s' %line)
        continue
        
    data= line.split('\t')
    entrezid=  data[0]
    if entrezid in goi:
        f_out.write('%s' %line)
        goi_found+=1
f_out.close()
fp.close()
print('%s of goi were found in table' %goi_found)

566 genes in genes of interest
272 of goi were found in table


In [5]:

"""
FIST RANKING OF GENES OF INTEREST 

This script ranks the GWAS gene matrix 


Required files:
filteredrankedlist: output file of filtered and annotated nodes from the previous part of the script
onlyGWASranked: output file, ranked GWAS genes 
"""

#
# RANKING -  list- based approach 
#
import pandas as pd
import numpy as np
import operator
#===============================================
#creating a dictionary with single genes and IDS
#
#===============================================
ID_symbol={}
symbol_ID={}
f=open('symbols2entrezIDs.txt','r')
for line in f:
    if line[0]=='#':
        continue
    data      = line.strip().split('\t')
    symbol    = data[0]
    ID        = data[1]
    ID_symbol[ID]=symbol
    symbol_ID[symbol]=ID
f.close()


#read in table with visitation probabilities 
what=pd.read_csv(filteredrankedlist, sep='\t')
mykeylist=[]
for column in what:
    if column =="entrez_id":
        mykeylist=list(what[column])
        
         

all_ranking_dict={}
for column in what:
    
    if column =="entrez_id":
        key_list=list(what[column])
        continue
    
    the_value_col=list(what[column])
    myset_length=len(set(the_value_col))
    i=0
    my_dict_vals={}
    while i< len(the_value_col):
        my_dict_vals[key_list[i]]=the_value_col[i]
        i+=1
    
    sorted_dict_vals = sorted(my_dict_vals.items(), key=operator.itemgetter(1),reverse= True )
    #print(sorted_dict_vals[1:10])

    new_ranking={}
    rankno=0
    i=0
    while i< len(the_value_col):
        mytup=sorted_dict_vals[i]
        if i==0:
            rankno+=1
            new_ranking[mytup[0]]=rankno
            i+=1
        else:
            
            currentval=mytup[1]
            previousval=sorted_dict_vals[i-1][1]
            prev_item=sorted_dict_vals[i-1][0]
            previousrank=new_ranking[prev_item]
            if currentval==previousval:
                new_ranking[mytup[0]]=previousrank
            else:
                rankno+=1
                new_ranking[mytup[0]]=rankno
            i+=1
            
        
    all_ranking_dict[column]=new_ranking
    
    
print(all_ranking_dict.keys())
                
    
f=open(onlyGWASranked,'w')
f.write('entrez_ID\tsymbol\t')

for item in all_ranking_dict.keys():
    mynewname=item.replace('prestartprob_','')
    mynewname+='_rank'
    f.write('%s\t' %mynewname)
f.write('\n')
for thekey in mykeylist: 
    
    try:
        mysim=ID_symbol[str(thekey)]
    except:
        mysim=thekey
    f.write('%s\t%s\t' %(thekey,mysim ))
    for myiteration, values in all_ranking_dict.items():
        f.write('%s\t' %values[thekey])
    f.write('\n')
        
f.close()



dict_keys(['visit_probab_r09', 'visit_probab_r08', 'visit_probab_r07', 'visit_probab_r06', 'visit_probab_r05', 'visit_probab_r04', 'visit_probab_r03', 'visit_probab_r02', 'visit_probab_r01'])


In [1]:

"""
ANNOTATING GENES OF INTEREST 


This script annotated the ranked genes of interest 


Required files:
filteredrankedlist: output file of filtered and annotated nodes from the previous part of the script
onlyGWASranked: input file, ranked GWAS genes from the previous step

onlyGWASranked_filtered: annotated and filtered table
"""

#
# filtering for known genes
#
#

#list of known gene defects 
import basic_file_things as bft
IUIS=bft.read_gene_file_to_one('iuis2017_entID.txt')
#orphanet=bft.read_gene_file_to_one('../data/ORPHANET_disease_genes_formatted.txt')
#combinedlist=set(IUIS)|set(orphanet)
combinedlist=set(IUIS)
f=open(onlyGWASranked,'r')
fp=open(onlyGWASranked_filtered,'w')

lineno=0
for line in f:
    if lineno==0:
        newline_0=line.replace('\n', '\tknowngene\n')
        fp.write('%s' %newline_0)
        lineno+=1
        continue
        
    data= line.split('\t')
    mygene=data[0]
    mysym=data[1]
    if mygene in combinedlist:
        newline=line.replace('\n', 'diseasegene\n')
        fp.write('%s' %newline)
        print('disease gene found: %s' %mysym)
    else:
        newline=line.replace('\n', 'novelgene\n')
        fp.write('%s' %newline)

f.close()
fp.close()




> done reading gene sets:
> 277 genes found
disease gene found: IL10
disease gene found: NCF4
disease gene found: CFB
disease gene found: CD25
disease gene found: CARD9
disease gene found: DNMT3B
disease gene found: TYK2
disease gene found: NFKB1
disease gene found: PTPRC
disease gene found: CSF2RB
disease gene found: IL7R
disease gene found: CIITA
disease gene found: CD40
disease gene found: TRAF3IP2
disease gene found: IKZF1
disease gene found: RASGRP1
disease gene found: IFNGR2
disease gene found: BACH2
disease gene found: RORC
disease gene found: IFIH1
